# Prompt tuning model training for T5 with encoder-decoder prompts to produce positive reviews

In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import string


import numpy as np
import statistics as st
import glob
import sys
import io

import zipfile
import tarfile
import logging
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")
import math

In [ ]:
#Set one cuda visible device if multiple GPUs are avialable
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
#!pip install transformers

In [ ]:
from transformers import (
    T5TokenizerFast,
    get_scheduler
)
import torch

from transformers.optimization import Adafactor, AdafactorSchedule

sys.path.append("..")

from model_classes.model_t5_encoder_decoder_prompt import T5PromptTuningLM


In [ ]:
# Fix the seed to be able to get the same randomness across runs and hence reproducible outcomes
def get_device_and_set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    return device
    
SEED = 123
device = get_device_and_set_seed(SEED)

In [ ]:
class Config:
    
    num_train_epochs = 20
    learning_rate = 0.15
    warmup_steps = 500
    max_train_steps = num_train_epochs
    weight_decay=0.01
    batch_size = 10
    # Prompt-tuning
    # number of prompt tokens
    n_prompt_tokens = 20
    # If True, soft prompt will be initialized from vocab 
    # Otherwise, you can set `random_range` to initialize by randomization.
    init_from_vocab = True
    # random_range = 0.5
args = Config()

# Training

In [ ]:
#model

In [ ]:
tokenizer = T5TokenizerFast.from_pretrained("google/t5-small-lm-adapt")
# Initialize GPT2LM with soft prompt


    
model = T5PromptTuningLM.from_pretrained(
    "google/t5-small-lm-adapt",
    n_tokens=args.n_prompt_tokens,
    initialize_from_vocab=args.init_from_vocab,
    device=device
).to(device)

In [ ]:
tokenizer.pad_token_id

In [ ]:
df_pos_train = pd.read_csv("../data/2_data_remove_duplicates_25_pos_sampled.csv", encoding='utf-8')
df_pos_val = pd.read_csv("../data/2_data_remove_duplicates_5_pos_sampled_val.csv", encoding='utf-8')
df_pos_test = pd.read_csv("../data/2_data_remove_duplicates_5_pos_sampled_test.csv", encoding='utf-8')


In [ ]:

train_texts = df_pos_train["reviewText"].tolist()
train_texts = shuffle(np.array(train_texts), random_state=SEED)

val_texts = df_pos_val["reviewText"].tolist() 
val_texts = shuffle(np.array(val_texts), random_state=SEED)

test_texts = df_pos_test["reviewText"].tolist() 
#test_texts = shuffle(np.array(test_texts), random_state=SEED)

del df_pos_train 
del df_pos_val 
del df_pos_test 


In [ ]:
class AmazonDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __getitem__(self, idx):

        encodings = self.tokenizer(self.texts[idx], truncation=True, padding="max_length")
        item = {key: torch.tensor(val) for key, val in encodings.items()}
        lst_attn_msk = item['attention_mask'].tolist()
        sent_end = lst_attn_msk.index(0) if 0 in lst_attn_msk else len(lst_attn_msk)
        
        item['labels'] = torch.tensor([x or -100 for x in item["input_ids"][1:]]).to(device)
        
        inp_1 = item["input_ids"][:(sent_end-2)]
        inp_2 = item["input_ids"][sent_end-1:]
        item['input_ids'] = torch.cat((inp_1, inp_2)).to(device)
        
        item['attention_mask'] = item['attention_mask'][1:].to(device)
        item['decoder_input_ids'] = item['input_ids'].to(device)
        item['decoder_attention_mask'] = item['attention_mask'].to(device)
        
        return item

    def __len__(self):
        return len(self.texts)
    
train_dataset = AmazonDataset(train_texts, tokenizer)
val_dataset = AmazonDataset(val_texts, tokenizer)


In [ ]:
from torch import nn
from transformers import Trainer
from transformers.trainer_utils import (
    PREFIX_CHECKPOINT_DIR,
    HPSearchBackend,
)


class T5Trainer(Trainer):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def compute_loss(self, model, inputs, return_outputs=False):
        model.to(device)
        model.set_device(device)
        labels = inputs.get("labels").to(device)
        if labels is not None:
            labels = model._extend_labels(labels).to(device)
        # forward pass
        
        outputs = model(**inputs)
        logits = outputs.get("logits").to(device)
        #print("logit shape = ", logits.shape)
        # compute custom loss 
        loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
        
        
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        #print("loss= ",loss)
        return (loss, outputs) if return_outputs else loss
    
    def _save_checkpoint(self, model, trial, metrics=None):
       # insert your own behavior here
    
        run_dir = self._get_output_dir(trial=trial)
        checkpoint_folder = f"{PREFIX_CHECKPOINT_DIR}-{self.state.global_step}"
        output_dir = os.path.join(run_dir, checkpoint_folder)
        
        model.save_soft_prompt(path=output_dir, filename="soft_prompt_T5_pos.model")
        
    def _get_output_dir(self, trial):
        if self.hp_search_backend is not None and trial is not None:
            if self.hp_search_backend == HPSearchBackend.OPTUNA:
                run_id = trial.number
            elif self.hp_search_backend == HPSearchBackend.RAY:
                from ray import tune

                run_id = tune.get_trial_id()
            elif self.hp_search_backend == HPSearchBackend.SIGOPT:
                run_id = trial.id
            elif self.hp_search_backend == HPSearchBackend.WANDB:
                import wandb

                run_id = wandb.run.id
            run_name = self.hp_name(trial) if self.hp_name is not None else f"run-{run_id}"
            run_dir = os.path.join(self.args.output_dir, run_name)
        else:
            run_dir = self.args.output_dir
        return run_dir

In [ ]:
len(train_dataset)

In [ ]:
from transformers import  Trainer, TrainingArguments



step_len = len(train_dataset)//args.batch_size


# Only update soft prompt'weights for prompt-tuning. ie, all weights in LM are set as `require_grad=False`. 
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if (n == "encoder_soft_prompt.weight" or n == "decoder_soft_prompt.weight")],
    }
]


optimizer = Adafactor(optimizer_grouped_parameters, lr=args.learning_rate,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=1e-5,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,)

lr_sceduler = AdafactorSchedule(optimizer, initial_lr=args.learning_rate)



training_args = TrainingArguments(
    output_dir='./results_mod_2_3',          # output directory
    num_train_epochs=args.num_train_epochs,              # total number of training epochs
    per_device_train_batch_size=args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=args.batch_size,   # batch size for evaluation
    warmup_steps=args.warmup_steps,                # number of warmup steps for learning rate scheduler
    logging_dir='./logs_2',            # directory for storing logs
    logging_steps=step_len//10,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    seed=SEED,
    
)



trainer = T5Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    optimizers = (optimizer, lr_sceduler),
    args=training_args,                  # training arguments, defined above
    tokenizer = tokenizer,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
)


trainer.args._n_gpu=1


In [ ]:
trainer.train()

# Inference

In [ ]:
#Inference

In [ ]:
tokenizer = T5TokenizerFast.from_pretrained("google/t5-small-lm-adapt")
# Load the model
model = T5PromptTuningLM.from_pretrained(
    "google/t5-small-lm-adapt",
    encoder_soft_prompt_path="../trained_models/t5_encoder_decoder/positive/encoder_soft_prompt_T5_pos.model",
    decoder_soft_prompt_path="../trained_models/t5_encoder_decoder/positive/decoder_soft_prompt_T5_pos.model",
    device=device
).to(device)
model.eval()
print(1)

In [ ]:
test = "the movie was"

call = tokenizer(test, return_tensors="pt").input_ids

beam_outputs = model.generate(
    input_ids=torch.tensor([call.tolist()[0][:-1]]).to(device), 
    decoder_input_ids=torch.zeros([1,1]).long().to(device), 
    min_length=200,
    max_length=200,
    num_beams=10,
    do_sample=True,
    no_repeat_ngram_size=1,  
    temperature = 1.0,
    top_k = 0,
    top_p = 0.8,
    repetition_penalty = 1.0,
    use_cache=False,
    early_stopping=True,
)

In [ ]:
print(tokenizer.decode(beam_outputs[0], skip_special_tokens=True))

In [ ]:
class AmazonDatasetTest(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, inp_perecentage=0.4):
        self.texts = texts
        self.tokenizer = tokenizer
        self.inp_perecentage = inp_perecentage

    def __getitem__(self, idx):

        encodings = self.tokenizer.encode(self.texts[idx], truncation=True, padding=True, return_tensors='pt')
        item = {}
        full_ids = encodings.tolist()[0][:-1]
        
        item["full_text"]=self.texts[idx]
        full_len = len(full_ids)
    
        input_len = math.floor(full_len*self.inp_perecentage)
        
        if input_len < 3:
            input_len=3
        elif input_len > 20:
            input_len=20
        
        if full_len < 25:
            full_len=25
            
        item["min_length"] = (input_len+full_len)+args.n_prompt_tokens+80
        item["max_length"] = (input_len+full_len)+args.n_prompt_tokens+80
        item["input_ids"] = torch.tensor([full_ids[:input_len]]).to(device)
        item["full_len"] = full_len
        item["input_len"] = input_len
        item["full_ids"]= full_ids
        
        return item

    def __len__(self):
        return len(self.texts)
    

test_dataset = AmazonDatasetTest(test_texts, tokenizer)

In [ ]:
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
from torch import nn

def bleu_score(li_abs_hyp, li_abs_ref):
    """
    Computes the BLEU score
    :param li_abs_hyp: list of hypothesis abstracts (token strings)
    :param li_abs_ref: list of reference abstracts (token strings)
    """
    bleu = corpus_bleu(li_abs_hyp, [li_abs_ref])

    return bleu.score


def rouge_score(li_abs_hyp, li_abs_ref):
    """
    Computes the ROUGE score
    :param li_abs_hyp: list of hypothesis abstracts (token strings)
    :param li_abs_ref: list of reference abstracts (token strings)
    """
    rouge_scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}

    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    for hyp, ref in zip(li_abs_hyp, li_abs_ref):
        local_rouge_scores = scorer.score(ref, hyp)
        for rouge_type in rouge_scores.keys():
            rouge_scores[rouge_type] += local_rouge_scores[rouge_type].fmeasure

    # Compute the averages 
    for rouge_type in rouge_scores.keys():
        rouge_scores[rouge_type] = rouge_scores[rouge_type] / max(len(li_abs_hyp), 1e-7)
    
    return rouge_scores

In [ ]:
def evaluate_T5_gpu_2(net, data_iter, device=None):
    """Compute the f1 score for a model on a dataset using a GPU.

    Defined in :numref:`sec_lenet`"""
    if isinstance(net, nn.Module):
        net.eval()  # Set the model to evaluation mode
        if not device:
            device = next(iter(net.parameters())).device
    # No. of correct predictions, no. of predictions

    
    y_tot =[]
    y_hat_tot = []
    y_hat_tot_plus = []
    input_texts_ls = []
    full_texts=[]
    perplexities = []

    with torch.no_grad():
        
        for inputs in tqdm(data_iter, total=len(data_iter)):
            
            beam_outputs = net.generate(
                input_ids=inputs["input_ids"], 
                decoder_input_ids=torch.zeros([1,1]).long().to(device),
                min_length=inputs["min_length"],
                max_length=inputs["max_length"],
                num_beams=10, 
                do_sample=True,
                no_repeat_ngram_size=1,  
                temperature = 1.0,
                top_k = 0,
                top_p = 0.8,
                repetition_penalty = 1.0,
                use_cache=False,
                early_stopping=True,
)
            
            
            y_hat = " ".join(data_iter.tokenizer.decode(beam_outputs[0], skip_special_tokens=True).split()[:(inputs["full_len"])]).lower()
            y_hat_plus = " ".join(data_iter.tokenizer.decode(beam_outputs[0], skip_special_tokens=True).split()[:(inputs["full_len"]+inputs["input_len"])]).lower()
            input_text = data_iter.tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True ).lower()
            
            y_hat = y_hat.replace(input_text, "").strip()
            y_hat_plus = " ".join(y_hat_plus.replace(input_text, "").strip().split()[:(inputs["full_len"])])
            
            #print(y_hat_plus,"\n")
            
            y_tot += [inputs["full_text"]]
            #full_texts += [inputs["full_text"]]
            input_texts_ls += [input_text]
            y_hat_tot += [y_hat]
            y_hat_tot_plus += [y_hat_plus]
            
            
            inps_2 = data_iter.tokenizer.encode(y_hat_plus)[:-1]
            
            labels_2 = torch.tensor([inps_2[1:][:511]+[1]]).to(device)
            inputs_2 = torch.tensor([inps_2[:-1][:511]+[1]]).to(device)
            mask_2 =  torch.tensor([[1]*inputs_2.shape[1]]).to(device)
        
            loss_2 = net(input_ids=inputs_2, attention_mask=mask_2, decoder_input_ids=inputs_2, decoder_attention_mask=mask_2 , labels=labels_2).get("loss").detach() 
            
            #loss_m = (loss_2 - loss_1) / (inputs_2.shape[1] - inputs_1.shape[1])
            ppl = math.exp(loss_2.item())
            if ppl < 1e4:   # for sanity
                perplexities.append(ppl)
                #print("added ppl = ", ppl)
            else:
                print("missed ppl = ", ppl)
                
            
            
    bleu_value = bleu_score(y_hat_tot_plus, y_tot)
    try:
        rouge_value = rouge_score(y_hat_tot_plus, y_tot)
    except:
        rouge_value = {"rouge1": 0.00}
            

    return bleu_value, rouge_value, y_hat_tot, y_hat_tot_plus, y_tot, input_texts_ls, np.nanmean(perplexities)


In [ ]:
bl2, rg2, predictions2, preds_plus2, full_texts2, input_texts2, ppl2 = evaluate_T5_gpu_2(model, test_dataset )

In [ ]:
def distinctness(generations):
    
    unigrams, bigrams, trigrams = set(), set(), set()
    total_words = 0
    for gen in generations:
        o = gen.split(' ')
        total_words += len(o)
        unigrams.update(o)
        for i in range(len(o) - 1):
            bigrams.add(o[i] + '_' + o[i+1])
        for i in range(len(o) - 2):
            trigrams.add(o[i] + '_' + o[i+1] + '_' + o[i+2])
    dist1 = (len(unigrams) / total_words)
    dist2 = (len(bigrams) / total_words)
    dist3 = (len(trigrams) / total_words)
    
    return dist1, dist2, dist3

In [ ]:
def get_metrics_2(y_hat_tot_plus, y_tot):
    
    bleu_value = bleu_score(y_hat_tot_plus, y_tot)
    try:
        rouge_value = rouge_score(y_hat_tot_plus, y_tot)
    except:
        rouge_value = {"rouge1": 0.00}
        
                
    return bleu_value, rouge_value